In [1]:
import nltk
import re
import string
import random
#import wikipedia
#ann = wikipedia.page("Artificial neural network")
#file = open('ann.txt', 'w')
#file.write(ann.content)
# used the wikipedia library to get plain text from wiki and i wrote it to ann.txt

In [2]:
file = open('ann.txt')
raw_txt = file.read()

In [3]:
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer

def normalize_text(text):
    word_tokens = word_tokenize(text)
    sent_tokens = sent_tokenize(text)
    word_tokens = [word for word in word_tokens if word.isalpha()]
    word_tokens = [word.lower() for word in word_tokens]
    word_tokens = [word for word in word_tokens if not word in stopwords.words("english")]

    lemma = WordNetLemmatizer()
    word_tokens = [lemma.lemmatize(word, pos = "v") for word in word_tokens]
    word_tokens = [lemma.lemmatize(word, pos = "n") for word in word_tokens]
    return [word_tokens, sent_tokens]

In [4]:
[word_tokens, sent_tokens] = normalize_text(raw_txt)

In [5]:
import random
def greet_user(user_input, greetings, responses):
    for greeting in greetings:
        if user_input.lower() in greeting.lower():
            return random.choice(responses)
    return 0
my_greetings = ["Hey Bobby", "Hello", "Hi", "How are you", "What's up", "Can I ask you something?"]
bot_responses = ["Hello Human", "Good day to you", "Hi", "How are you today", "A sunny day today huh?"]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def do_calcs(input):
    TfidfVec = TfidfVectorizer()
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    
    response = ''
    if(req_tfidf == 0):
        response = "I am sorry, I do not understand what you are asking me."
        return response
    response = sent_tokens[idx]
    return response

In [7]:
def respond(user_input, word_tokens, sent_tokens):
    if(user_input in ["goodbye thanks", "goodbye", "bye", "thanks", "thank you goodbye", "thank you", "take care", "bye bobby", "goodbye bobby"]):
        return 0
    sent_tokens.append(user_input)
    word_tokens = word_tokens + nltk.word_tokenize(user_input)
    final_words = list(set(word_tokens))
    res = do_calcs(user_input)
    sent_tokens.remove(user_input)
    return res

In [9]:
import speech_recognition as sr

def speech_to_text(audio_path):
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
        try:
            return r.recognize_google(audio)
        except sr.UnknownValueError:
            return -1
        except sr.RequestError as e:
            return -1

In [10]:
def mic_speech_to_text():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print("...") # to prompt user for input
        audio = r.listen(source)
        try:
            return r.recognize_google(audio)
        except sr.UnknownValueError:
            return "xxxxxxxzzzzzz" # on purpose so that the bot doesnt understand
        except sr.RequestError as e:
             return "xxxxxxxzzzzzz" # on purpose so that the bot doesnt understand

In [11]:
audio_paths = ["audio-files/hey_bobby.wav", "audio-files/what_is_ann.wav", "audio-files/thankyou.wav"]
audio_txt = [speech_to_text(audio_path) for audio_path in audio_paths]

In [15]:
from gtts import gTTS
import os
from playsound import playsound

def read_output(output):
    language = "en"
    speech = gTTS(text = output, lang = language, slow = False)
    speech.save("output.mp3")
    playsound("output.mp3")

playsound is relying on a python 2 subprocess. Please use `pip3 install PyObjC` if you want playsound to run more efficiently.


In [17]:
print("Hello, my name is Bobbyette Bot! Ask me anything about Artifical Neural Networks!")

print("But first please indicate your input device:\n 0- Keyboard\n 1- Microphone")
input_type = input()

print("\n")

print("Please indicate your output format:\n 0- Text\n 1- Text & Sound")
output_type = input()

# greet_user(audio_txt[0], my_greetings, bot_responses) # for transcribed audio
# i = 1  # for transcribed audio

initial_prompt = "Now ask me anything about Artificial Neural Networks!"
if(int(output_type) == 1):
    read_output(initial_prompt)
    print("Bobbyette: " + initial_prompt)
else:
    print("Bobbyette: " + initial_prompt) # default output is text
    
if(int(input_type) == 1):
    user_input = mic_speech_to_text() 
else:
    user_input = input() # default input device is keyboard

dnu_text = "I am sorry, I do not understand what you are asking me."
bot_greeting = greet_user(user_input, my_greetings, bot_responses)

if(bot_greeting == 0):
    if(int(output_type) == 1):
        print("Bobbyette: " + dnu_text)
        read_output(dnu_text)
    else:
        print("Bobbyette: " + dnu_text)
else:
    if(int(output_type) == 1):
        print("Bobbyette: " + bot_greeting)
        read_output(bot_greeting)
    else:
        print("Bobbyette: " + bot_greeting)

while(1):

    # user_input = audio_txt[i] # for transcribed audio
    # i += 1 # for transcribed audio
    
    if(int(input_type) == 1):
        user_input = mic_speech_to_text()
    else:
        user_input = input()
        
    bot_greeting = greet_user(user_input, my_greetings, bot_responses) # assuming multiple greetings during same session
    if(bot_greeting == 0):
        bot_response = respond(user_input, word_tokens, sent_tokens)
        if(bot_response == 0):
            if(int(output_type) == 1):
                print("Bobbyette: " + "Goodbye Human!")
                read_output("Goodbye Human!")
            else:
                print("Bobbyette: " + "Goodbye Human!")
            break
        else:
            if(int(output_type) == 1):
                print("Bobbyette: " + bot_response)
                read_output(bot_response) 
            else:
                print("Bobbyette: " + bot_response)
    else: 
        if(int(output_type) == 1):
            print("Bobbyette: " + bot_greeting)
            read_output(bot_greeting)
        else:
            print("Bobbyette: " + bot_greeting)
    

Hello, my name is Bobbyette Bot! Ask me anything about Artifical Neural Networks!
But first please indicate your input device:
 0- Keyboard
 1- Microphone
0


Please indicate your output format:
 0- Text
 1- Text & Sound
0
Bobbyette: Now ask me anything about Artificial Neural Networks!
what are ann
Bobbyette: I am sorry, I do not understand what you are asking me.
what are artificial neural networks
Bobbyette: Artificial neural networks (ANNs), usually simply called neural networks (NNs), are computing systems vaguely inspired by the biological neural networks that constitute animal brains.
what is deep learning
Bobbyette: === Practical counterexamples ===
Analyzing what has been learned by an ANN, is much easier than to analyze what has been learned by a biological neural network.


KeyboardInterrupt: Interrupted by user